In [1]:
import os
import sys
import numpy as np

# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)


from Dataset.cancer_dataset_for_LS import main_generate_cancer_matrices_for_LS, load_cancer_dataset_matrices_for_LS
from analysis.commons import data_normalize_by_features, concatenate_B_b

from estimator.NDIS import BasicNDISEstimator
from analysis.RP_privacy_analysis_advanced import gaussian_projection_ndis_delta, rp_mean_cov_matrix
from analysis.RP_privacy_analysis_advanced import isotropic_gaussian_scaling

In [4]:
file_X_path = os.path.join(project_dir, "Dataset", "cancer-LR-X.txt")
file_y_path = os.path.join(project_dir, "Dataset", "cancer-LR-y.txt")

X = None
y = None
try:
    X, y = load_cancer_dataset_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
except:
    main_generate_cancer_matrices_for_LS(file_X_name=file_X_path, file_y_name=file_y_path)
    X, y = load_cancer_dataset_matrices_for_LS()

B, b = data_normalize_by_features(X, y)
A = concatenate_B_b(B, b)

In [5]:
x = B.copy()
r = 2
eps = 1
index = 186

gaussian_projection_ndis_delta(x, index, r, eps)


0.10026828161999324

In [4]:
mean_1, _, cov_1 = rp_mean_cov_matrix(x, r, full_vec_cov=True)

x_prime = x.copy()
x_prime[index, :] = 0
mean_2, _, cov_2 = rp_mean_cov_matrix(x_prime, r, full_vec_cov=True)

estimator = BasicNDISEstimator(mean_1.ravel(), cov_1, mean_2.ravel(), cov_2, workers=50, num_samples=2**16)

method = "pqmc"
print(f"delta of ALS for index {index}: {estimator.estimate(eps, method=method)} with epsilon = {eps}")

delta of ALS for index 186: {'delta': 0.10026828518098971, 'se': 3.095769517082433e-09} with epsilon = 1


In [14]:
r = 10
mean_1 = np.zeros(r)
cov_1 = np.eye(r)
t = 0.3 
mean_2 = np.zeros(r)
cov_2 = (t**2)*cov_1

estimator = BasicNDISEstimator(mean_1.ravel(), cov_1, mean_2.ravel(), cov_2, workers=50, num_samples=2**16)

method = "pqmc"
print(f"delta of ALS for index {index}: {estimator.estimate(eps, method=method)} with epsilon = {eps}")

delta of ALS for index 186: {'delta': 0.9857120776452256, 'se': 5.6012916233208456e-05} with epsilon = 1


In [15]:
isotropic_gaussian_scaling(t, r, eps)

0.9858235871266626